In [6]:
from imputationLibrary import decompose, forwardFilling, hotDeck, meanImputation, movingAverage, splineInterpolation, randomSampleImputation, nature
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from sklearn import preprocessing
from datetime import datetime, timedelta
from tsmoothie.smoother import *

In [7]:
def plot_ac(df, name):
    normalized = preprocessing.scale([np.array(df.fillna(0))])
    corr = signal.correlate(normalized, normalized, mode='full')
    plt.plot(corr[0], 'o-', markersize=2)
    plt.axhline(2/np.sqrt(len(df)), ls=':')
    plt.axhline(-2/np.sqrt(len(df)), ls=':')
    plt.title(name + ' auto-correlation')
    plt.show()

In [8]:
def decompose_and_plot(ts, flag_plot = False):
    ts_decomposed = decompose.additive(ts, period)
    if flag_plot:
        ts_decomposed.plot()
        plt.show()
        print("Resid is white noise? ", nature.isWhiteNoise(ts_decomposed.resid))
        print("Resid is seasonal noise? ", nature.isSeasonal(ts_decomposed.resid))
        print("Resid is trended noise? ", nature.isTrended(ts_decomposed.resid, period))
        print("Resid is seasonal and trended noise? ", nature.isTrendedAndSeasonal(ts_decomposed.resid, period))
        plot_ac(ts_decomposed.resid, "Resid")
        plot_ac(ts_decomposed.trend, "Trend")
        plot_ac(ts_decomposed.seasonal, "Seasonal")
    return ts_decomposed.resid, ts_decomposed.trend, ts_decomposed.seasonal
    

In [9]:
df = pd.read_csv(r'output\/CompleteCovid.csv',index_col=0, parse_dates=True)
df_train = df.loc['2020-01-22 09:00:00':'2020-02-26 02:00:00'].dropna(how='all', axis=1) #Selecting training data
df_test = df.loc['2020-02-26 02:30:00':, df_train.columns] #Selecting test data
period=30
df_filled = df_train.fillna(0)
df_filled_test = df_test.fillna(0)

df_decomposed_resid = pd.DataFrame()
df_decomposed_trend = pd.DataFrame()
df_decomposed_seasonal = pd.DataFrame()

df_decomposed_resid_test = pd.DataFrame()
df_decomposed_trend_test = pd.DataFrame()
df_decomposed_seasonal_test = pd.DataFrame()

df_nan = df_train.isna()
df_nan_test = df_test.isna()

decomp_type_dict_train = {}
decomp_type_dict_test = {}
for col in df_filled.columns:
    resid, trend, seasonal = decompose_and_plot(df_filled.loc[:,col])
    df_decomposed_resid[col] = resid
    df_decomposed_trend[col] = trend
    df_decomposed_seasonal[col] = seasonal

for col in df_filled_test.columns:
    resid, trend, seasonal = decompose_and_plot(df_filled_test.loc[:,col])
    df_decomposed_resid_test[col] = resid
    df_decomposed_trend_test[col] = trend
    df_decomposed_seasonal_test[col] = seasonal

df_final_resid = df_decomposed_resid.mask(df_nan == True, np.nan)
df_final_trend = df_decomposed_trend.mask(df_nan == True, np.nan)
df_final_seasonal = df_decomposed_seasonal.mask(df_nan == True, np.nan)

df_final_resid_test = df_decomposed_resid_test.mask(df_nan_test == True, np.nan)
df_final_trend_test = df_decomposed_trend_test.mask(df_nan_test == True, np.nan)
df_final_seasonal_test = df_decomposed_seasonal_test.mask(df_nan_test == True, np.nan)

df_white_noise_train, df_white_noise_test =  meanImputation.input(df_final_resid, df_final_resid_test)
df_seasonal_train, df_seasonal_test = splineInterpolation.input(df_final_seasonal, df_final_seasonal_test)
df_trended_train, df_trended_test = forwardFilling.input(df_final_trend, df_final_trend_test)

df_final_train = df_white_noise_train + df_seasonal_train + df_trended_train
df_final_test = df_white_noise_test + df_seasonal_test + df_trended_test

### USE KALMAN FILTER TO SMOOTH ALL DATA (ONLY VISUALIZATION PURPOSE) ###

smoother_train = KalmanSmoother(component='level_longseason', 
                          component_noise={'level':0.1, 'longseason':0.1}, 
                          n_longseasons=365)
smoother_train.smooth(df_final_train.T)

smoother_test = KalmanSmoother(component='level_longseason', 
                          component_noise={'level':0.1, 'longseason':0.1}, 
                          n_longseasons=365)
smoother_test.smooth(df_final_test.T)

df_filled = pd.DataFrame(data = smoother_train.smooth_data.T, index = df_train.index, columns= df_train.columns)
df_filled_test = pd.DataFrame(data = smoother_test.smooth_data.T, index = df_test.index, columns= df_test.columns)


df_filled.to_csv(r'output\/CompleteCovidTrainAddDecompKalman_0.csv', index = True)
df_filled_test.to_csv(r'output\/CompleteCovidTestAddDecompKalman_0.csv', index = True)

In [10]:
df_filled

Asia_confirmed_cases  Asia_deaths  Asia_recovered  \
update_time                                                              
2020-01-22 09:00:00           1542.936596    38.158827      215.101408   
2020-01-22 09:30:00           1703.445829    42.537796      238.381401   
2020-01-22 10:00:00           1937.957893    48.899111      272.064638   
2020-01-22 10:30:00           2197.287096    55.927338      308.977277   
2020-01-22 11:00:00           2451.135505    62.811800      344.719278   
2020-01-22 11:30:00           2681.291343    69.064884      376.665566   
2020-01-22 12:00:00           2877.239060    74.404799      403.321356   
2020-01-22 12:30:00           3033.305123    78.679413      423.903032   
2020-01-22 13:00:00           3146.769974    81.815955      438.060925   
2020-01-22 13:30:00           3216.569437    83.786561      445.688682   
2020-01-22 14:00:00           3242.326429    84.582774      446.781189   
2020-01-22 14:30:00           3223.519783    84.193918      441.312693   
2020-01-22 15:00:00           3158.624312    82.585030      429.110803   
2020-01-22 15:30:00           3044.050533    79.669949      409.701239   
2020-01-22 16:00:00           2872.672480    75.274181      382.092329   
2020-01-22 16:30:00           2631.649838    69.080146      344.456281   
2020-01-22 17:00:00           2299.109836    60.543868      293.643596   
2020-01-22 17:30:00           2346.807017    61.504256      298.705478   
2020-01-22 18:00:00           2310.160862    60.312219      291.736360   
2020-01-22 18:30:00           2218.137790    57.699550      277.018261   
2020-01-22 19:00:00           2090.494974    54.167251      257.482650   
2020-01-22 19:30:00           1940.966765    50.065562      235.174540   
2020-01-22 20:00:00           1779.249305    45.643913      211.539689   
2020-01-22 20:30:00           1612.179365    41.080752      187.592235   
2020-01-22 21:00:00           1444.342615    36.499224      163.996194   
2020-01-22 21:30:00           1278.232708    31.971795      141.077107   
2020-01-22 22:00:00           1113.992923    27.514784      118.766133   
2020-01-22 22:30:00            948.688811    23.071673       96.465463   
2020-01-22 23:00:00            898.269903    21.448448       93.495911   
2020-01-22 23:30:00            847.277199    19.845247       90.802470   
...                                   ...          ...             ...   
2020-02-25 11:30:00           1868.391684    55.502946      656.207998   
2020-02-25 12:00:00           2225.825577    67.077572      743.848659   
2020-02-25 12:30:00           2702.870620    81.959777      863.151704   
2020-02-25 13:00:00           3466.635169   109.526718      981.521071   
2020-02-25 13:30:00           4329.637553   142.117259     1174.192438   
2020-02-25 14:00:00           5306.712512   182.180524     1478.106183   
2020-02-25 14:30:00           6513.215953   235.438062     1949.459250   
2020-02-25 15:00:00           8964.208975   332.201036     2805.103672   
2020-02-25 15:30:00           6379.433086   236.395429     2090.226689   
2020-02-25 16:00:00           4691.912308   173.933879     1625.204614   
2020-02-25 16:30:00           3617.457355   133.213242     1306.531396   
2020-02-25 17:00:00           2924.820989   105.508254     1065.997382   
2020-02-25 17:30:00           2422.009469    84.104567      858.408699   
2020-02-25 18:00:00           1950.426676    63.833028      653.899231   
2020-02-25 18:30:00           1385.682434    40.912309      433.301098   
2020-02-25 19:00:00           1076.819167    26.072552      288.212341   
2020-02-25 19:30:00            654.799597     8.268434      136.627644   
2020-02-25 20:00:00            202.699107    -9.554009       -1.832076   
2020-02-25 20:30:00           -183.603602   -24.071606     -106.671541   
2020-02-25 21:00:00           -413.333941   -32.185583     -159.675670   
2020-02-25 21:30:00           -419.747333   -31.667867     -148.519119   
2020-02-25 22:00:00      

In [11]:
df_filled_test

Asia_confirmed_cases    Asia_deaths  Asia_recovered  \
update_time                                                                
2020-02-26 02:30:00         7.493694e+191  2.509729e+190   1.363150e+191   
2020-02-26 03:00:00         4.681814e+191  1.567996e+190   8.516512e+190   
2020-02-26 03:30:00         2.875224e+191  9.629472e+189   5.230212e+190   
2020-02-26 04:00:00         1.714057e+191  5.740584e+189   3.117977e+190   
2020-02-26 04:30:00         9.672549e+190  3.239453e+189   1.759497e+190   
2020-02-26 05:00:00         4.864635e+190  1.629225e+189   8.849075e+189   
2020-02-26 05:30:00         1.764295e+190  5.908836e+188   3.209362e+189   
2020-02-26 06:00:00        -2.400633e+189 -8.040009e+187  -4.366901e+188   
2020-02-26 06:30:00        -1.541123e+190 -5.161406e+188  -2.803398e+189   
2020-02-26 07:00:00        -2.391015e+190 -8.007797e+188  -4.349405e+189   
2020-02-26 07:30:00        -2.951627e+190 -9.885355e+188  -5.369194e+189   
2020-02-26 08:00:00        -3.326904e+190 -1.114220e+189  -6.051846e+189   
2020-02-26 08:30:00        -3.583588e+190 -1.200187e+189  -6.518770e+189   
2020-02-26 09:00:00        -3.764534e+190 -1.260788e+189  -6.847922e+189   
2020-02-26 09:30:00        -3.897260e+190 -1.305239e+189  -7.089359e+189   
2020-02-26 10:00:00        -3.999437e+190 -1.339460e+189  -7.275224e+189   
2020-02-26 10:30:00        -4.082412e+190 -1.367249e+189  -7.426161e+189   
2020-02-26 11:00:00        -4.153474e+190 -1.391048e+189  -7.555427e+189   
2020-02-26 11:30:00        -4.217304e+190 -1.412426e+189  -7.671538e+189   
2020-02-26 12:00:00        -4.276909e+190 -1.432388e+189  -7.779964e+189   
2020-02-26 12:30:00        -4.334222e+190 -1.451583e+189  -7.884221e+189   
2020-02-26 13:00:00        -4.390486e+190 -1.470427e+189  -7.986568e+189   
2020-02-26 13:30:00        -4.446500e+190 -1.489186e+189  -8.088460e+189   
2020-02-26 14:00:00        -4.502777e+190 -1.508034e+189  -8.190832e+189   
2020-02-26 14:30:00        -4.559649e+190 -1.527082e+189  -8.294286e+189   
2020-02-26 15:00:00        -4.617330e+190 -1.546400e+189  -8.399212e+189   
2020-02-26 15:30:00        -4.675958e+190 -1.566035e+189  -8.505859e+189   
2020-02-26 16:00:00        -4.735622e+190 -1.586017e+189  -8.614392e+189   
2020-02-26 16:30:00        -4.796380e+190 -1.606366e+189  -8.724915e+189   
2020-02-26 17:00:00        -4.858270e+190 -1.627093e+189  -8.837496e+189   
...                                   ...            ...             ...   
2020-03-11 09:00:00        -1.198368e+193 -4.013478e+191  -2.179905e+192   
2020-03-11 09:30:00        -9.413514e+192 -3.152699e+191  -1.712377e+192   
2020-03-11 10:00:00        -5.862366e+192 -1.963377e+191  -1.066401e+192   
2020-03-11 10:30:00        -6.122985e+191 -2.050662e+190  -1.113809e+191   
2020-03-11 11:00:00         7.393178e+192  2.476064e+191   1.344865e+192   
2020-03-11 11:30:00         1.975991e+193  6.617831e+191   3.594450e+192   
2020-03-11 12:00:00         3.896351e+193  1.304935e+192   7.087705e+192   
2020-03-11 12:30:00         6.896840e+193  2.309835e+192   1.254578e+193   
2020-03-11 13:00:00         1.161439e+194  3.889798e+192   2.112729e+193   
2020-03-11 13:30:00         1.906600e+194  6.385434e+192   3.468225e+193   
2020-03-11 14:00:00         3.061415e+194  1.025305e+193   5.568905e+193   
2020-03-11 14:30:00         4.793405e+194  1.605369e+193   8.719502e+193   
2020-03-11 15:00:00         7.511186e+194  2.515587e+193   1.366332e+194   
2020-03-11 15:30:00         1.217783e+195  4.078504e+193   2.215224e+194   
2020-03-11 16:00:00         2.073487e+195  6.944359e+193   3.771803e+194   
2020-03-11 16:30:00         3.672151e+195  1.229848e+194   6.679872e+194   
2020-03-11 17:00:00         6.621269e+195  2.217543e+194   1.204450e+195   
2020-03-11 17:30:00         1.192631e+196  3.994264e+194   2.169470e+195   
2020-03-11 18:00:00         2.121452e+196  7.105000e+194   3.859054e+195   
2020-03-11 18:30:00         3.708285e+196  1.241949e+195   6.745601e+